Frame 顯示 使用 QLabel 類別

In [ ]:
# -*- coding: utf-8 -*-
from PyQt5 import QtWidgets, QtGui, QtCore
from PyQt5.QtWidgets import QMessageBox
from PyQt5.QtGui import QImage, QPixmap

from myShowCamera_ui import Ui_MainWindow
import sys
import os

import cv2

# 設定 camera 解析度
video_width = 640
video_height = 480

cap = cv2.VideoCapture(0)   # device number 0
cap.set(cv2.CAP_PROP_FRAME_WIDTH, video_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, video_height)
print("capture device is open: " + str(cap.isOpened()))

capture_output_path = "./Capture_Screen_Shot/"
if not os.path.isdir(capture_output_path):
    os.mkdir(capture_output_path)

In [ ]:
class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)
        
        self.logic=0
        self.value=1
        #============= 物件 初始化 設定=============
        #============= Set Window Icon ============
        self.setWindowIcon(QtGui.QIcon('camera_icon.png'))

        
        
        
        #============= 事件 功能 設計 =============
        # Menu
        self.ui.retranslateUi(self)
        self.ui.actionExit.setShortcut('Ctrl+Q')
        self.ui.actionExit.triggered.connect(self.close_and_release)
        
        # 呼叫 buttonCamera 
        self.ui.buttonCamera.clicked.connect(self.Camera_Capture)

        # 呼叫 buttonCapture 
        self.ui.buttonCapture.clicked.connect(self.CaptureClicked)
     
        
            
            
    def CaptureClicked(self):
        self.logic = 2
    
    #釋放 cap 和 關閉程式
    def close_and_release(self):
        self.close()
        
        # release VideoWriter object clsoe all windows
        cap.release()
        cv2.destroyAllWindows()
        
        
        
        
    #將 影像 顯示 在 GUI QLabel
    def displayImage(self, img, window=1):
        qformat = QImage.Format_Indexed8

        if len(img.shape) == 3:
            if(img.shape[2]) == 4:
                qformat = QImage.Format_RGBA888
            else:
                qformat = QImage.Format_RGB888

        img = QImage(img, img.shape[1], img.shape[0], qformat)
        img = img.rgbSwapped()

        self.ui.LabelFrame.setPixmap(QPixmap.fromImage(img))
        self.ui.LabelFrame.setAlignment(QtCore.Qt.AlignHCenter | QtCore.Qt.AlignVCenter)
    
    
    
    #Camera button 功能
    def Camera_Capture(self):
        self.ui.labelInfo.setText("Start camera !")
        self.ui.buttonCamera.setEnabled(False)
        
        # loop to read image  
        while True:

            # keyboard input value
            key = cv2.waitKey(1) & 0xFF

            # read from camera
            ret, frame = cap.read()

            if ret:  # success read
                self.displayImage(frame,1)
                #cv2.imshow('frame', frame)
                
                #Capture button 按下時，執行以下程式
                if (self.logic==2):
                    cv2.imwrite(capture_output_path + 'Image%s.png'%(self.value), frame)

                    self.logic=1
                    self.ui.labelInfo.setText('Image%s.png'%(self.value) + ' saved in ' + capture_output_path + ' folder !')
                    self.value=self.value+1
    
                
                    
                    
            else:
                print('camera read fail.')
                break

            if key == ord('q'):     # press 'q' to leave while
                break



In [ ]:
if __name__ == '__main__':
    app = QtWidgets.QApplication([])
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())
